In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime as dt

In [18]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

learning_rate = 0.01
n_epochs = 4
batch_size = 50
tf.reset_default_graph()

In [19]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")

In [20]:
def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = "weights")
        b = tf.Variable(tf.zeros([n_neurons]), name = "biasses")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

In [21]:
now = dt.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "./CH10_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

with tf.name_scope("dnn"): 
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation = "relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation = "relu")
    logits = neuron_layer(hidden2, n_output, "outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

acc_train_summary = tf.summary.scalar('Train_accuracy', accuracy)
acc_test_summary = tf.summary.scalar('Test_accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [13]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [22]:
with tf.Session()as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            _, acc_train = sess.run([training_op, accuracy], feed_dict = {X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict = {X: mnist.test.images, y : mnist.test.labels})
            if iteration % 10 == 0 :
                acc_train_str = acc_train_summary.eval(feed_dict = {X : X_batch, y : y_batch})
                acc_test_str = acc_test_summary.eval(feed_dict = {X : mnist.test.images, y : mnist.test.labels})
                step = epoch * batch_size + iteration
                file_writer.add_summary(acc_train_str, step)
                file_writer.add_summary(acc_test_str, step)
        print(epoch, " Train accuracy ", acc_train, " Test_accuracy ", acc_test)
    save_path = saver.save(sess,"./CH10/my_model_final.ckpt")
file_writer.close()

0  Train accuracy  0.88  Test_accuracy  0.9148
1  Train accuracy  0.94  Test_accuracy  0.9336
2  Train accuracy  1.0  Test_accuracy  0.9392
3  Train accuracy  0.94  Test_accuracy  0.9482
